In [ ]:
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from evaluate.lane import LaneEval
import random
import os
import pickle
%matplotlib inline

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D, Dense, Flatten
from keras.layers import Deconvolution2D, Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


# Load Data

In [ ]:
#Load trainig images
pickle_data = pickle.load(open("GT_images.p", "rb" ))
train_images = pickle_data["GroundTruth"]

#Load Polinomials
pickle_data = pickle.load(open("polynomials.p", "rb" ))
labels = pickle_data["Polynomials"]
plt.imshow(train_images[70])
plt.show()

new_labels = []
for i in range(len(train_images)):
    train_images[i]=np.float32(train_images[i][:,:,:3])
    mask = train_images[i][:,:,1]
    train_images[i]=mask
    
    coeffs=np.ndarray.tolist(np.concatenate(np.asarray(labels[i])))
    #Five lanes included, triming :/ this could be done better
    if(len(coeffs)>12):
        coeffs=coeffs[:12]
    new_labels.append(coeffs)
    

train_images=np.float32(train_images).reshape((-1,80,160,1))
new_labels=np.asarray(new_labels)
labels=[]
print("Training data Loaded")
print(train_images.shape,new_labels.shape)


# Data Conditioning

In [ ]:

#Scaler for polynomial coeffs
Y_scaler=StandardScaler().fit(new_labels)
Y_scaled=Y_scaler.transform(new_labels)
#Image normalization
X=train_images/255

# Shuffle images along with their labels, then split into training/validation sets
X, Y_scaled = shuffle(X, Y_scaled)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(X, Y_scaled, test_size=0.15)

print("Training on: ", len(X_train), " samples")
print("Training on: ", len(X_val), " samples")

print(Y_scaled[0])

scaler_file= 'Y_scaler.sav'
pickle.dump(Y_scaler, open(scaler_file, 'wb'))
print("Scaler Saved")


In [ ]:
new_labels=[]
train_images=[]

# Model parameters


In [ ]:
batch_size = 150
epochs = 50
pool_size = (2, 2)
input_shape = X.shape[1:]
print(input_shape)

In [ ]:
model = Sequential()
# Normalizes incoming inputs. First layer needs the input shape to work
model.add(BatchNormalization(input_shape=input_shape))

# Below layers were re-named for easier reading of model summary; this not necessary
# Conv Layer 1
model.add(Convolution2D(2, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv1'))

# Conv Layer 2
#model.add(Convolution2D(3, 3, 3, border_mode='valid', subsample=(1,1), activation = 'relu', name = 'Conv2'))

# Pooling 1
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(12, activation= 'elu'))
model.add(Dense(12, activation = 'sigmoid'))

model.summary()



# Data Feeding

In [ ]:
datagen = ImageDataGenerator()
datagen.fit(X_train)

In [ ]:
opt=optimizers.Adam()
model.compile(optimizer=opt, loss='mean_absolute_percentage_error')
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), samples_per_epoch = len(X_train),
                    nb_epoch=epochs, verbose=1, validation_data=(X_val, y_val))

# Save model architecture and weights
model_json = model.to_json()
with open("lanes_CNN_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('lanes_CNN_model.h5')

print("Model Saved")